In [1]:
import numpy as np
import pylab
import pynbody
import pynbody.plot.sph as sph
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd

from astropy import units
from astropy import constants as c

import warnings
warnings.filterwarnings("ignore")

# from analysis import * 

In [8]:
def read_tracked_particles(sim, haloid, verbose=False):
    
    if verbose: print(f'Loading tracked particles for {sim}-{haloid}...')
    
    key = f'{sim}_{str(int(haloid))}'

    # import the tracked particles dataset
    path1 = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/tracked_particles_v2.hdf5'
    data = pd.read_hdf(path1, key=key)
    
    time = np.unique(data.time)
    dt = time[1:]-time[:-1]
    dt = np.append(dt[0], dt)
    dt = dt[np.unique(data.time, return_inverse=True)[1]]
    data['dt'] = dt
    
    
    if verbose: print('Successfully loaded')
    
    r_gal = np.array([])
    for t in np.unique(data.time):
        d = data[data.time==t]
        r_gas = np.mean(d.sat_r_gas)
        r_half = np.mean(d.sat_r_half)
        rg = np.max([r_gas,r_half])

        if np.isnan(rg):
            rg = r_gal_prev

        if verbose: print(f't = {t:1f} Gyr, satellite R_gal = {rg:.2f} kpc')
        r_gal = np.append(r_gal,[rg]*len(d))

        r_gal_prev = rg

    data['r_gal'] = r_gal
    
    r_gal_prev = 0
    r_gal = np.array([])
    for t in np.unique(data.time):
        d = data[data.time==t]
        r_gas = np.mean(d.host_r_gas)
        r_half = np.mean(d.host_r_half)
        rg = np.max([r_gas,r_half])

        if np.isnan(rg):
            rg = r_gal_prev

        if verbose: print(f't = {t:1f} Gyr, host R_gal = {rg:.2f} kpc')
        r_gal = np.append(r_gal,[rg]*len(d))

        r_gal_prev = rg

    data['host_r_gal'] = r_gal
    
    thermo_disk = (np.array(data.temp) < 1.2e4) & (np.array(data.rho) > 0.1)
    
    in_sat = np.array(data.in_sat)
    other_sat = np.array(data.in_other_sat)
    in_host = np.array(data.in_host) & ~in_sat & ~other_sat
    
    sat_disk = in_sat & thermo_disk
    sat_halo = in_sat & ~thermo_disk
    
    host_disk = in_host & thermo_disk
    host_halo = in_host & ~thermo_disk
    
    IGM = np.array(data.in_IGM)
    
    # basic location classifications.
    data['sat_disk'] = sat_disk
    data['sat_halo'] = sat_halo
    data['host_disk'] = host_disk
    data['host_halo'] = host_halo
    data['other_sat'] = other_sat
    data['IGM'] = IGM

    return data

In [11]:
key = 'h148_13'

path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
discharged = pd.read_hdf(path, key=key)

path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/dsrg_accreted_particles.hdf5'
accreted = pd.read_hdf(path, key=key)

path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/tracked_particles_v2.hdf5'
tracked = read_tracked_particles('h148', 13, verbose=False)

In [13]:
discharged[discharged.pid==2202002]
accreted[accreted.pid==2202002]

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,r_gal,host_r_gal,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM,t,sat_Mvir


In [5]:
import tqdm

dsrg_accreted = pd.DataFrame() # gas accreted following a discharge event.


# picking out all unique particles that were tracked in a the dwarf galaxy.
pids = np.unique(tracked.pid) 
dsrg_pids = np.unique(discharged.pid)




for pid in tqdm.tqdm(dsrg_pids): # iterating over each unique pid.
    data = tracked[tracked.pid==pid] # picking out all instances of the particular particle (each has same # of timesteps).
    dsrg_data = discharged[discharged.pid==pid]


    sat_disk = np.array(data.sat_disk, dtype=bool)
    in_sat = np.array(data.in_sat, dtype=bool)
    outside_disk = ~sat_disk

NameError: name 'tracked' is not defined

In [ ]:
def calc_accreted(sim, haloid, save=True, verbose=True):
    ''' 
    Computing accreted gas particles using method 1: a particle is recorded as accreted 
    if it was present in the 'discharged' dataset prior to the accretion event.
    '''
    
    import tqdm
    
    tracked = read_tracked_particles(sim, haloid, verbose=verbose)# all gas at all recorded times.
    discharged = read_discharged(sim, haloid, verbose=verbose) # all gas ejected from the disk (can include repeat events).
    
#     if verbose: print(f'Now computing discharged particles for {sim}-{haloid}...')
    dsrg_accreted = pd.DataFrame() # gas accreted following a discharge event.
    
    
    # picking out all unique particles that were tracked in a the dwarf galaxy.
    pids = np.unique(tracked.pid) 
    dsrg_pids = np.unique(discharged.pid)
    
    
    
    
    for pid in tqdm.tqdm(dsrg_pids): # iterating over each unique pid.
        data = tracked[tracked.pid==pid] # picking out all instances of the particular particle (each has same # of timesteps).
        dsrg_data = discharged[discharged.pid==pid]
    
        
        sat_disk = np.array(data.sat_disk, dtype=bool)
        in_sat = np.array(data.in_sat, dtype=bool)
        outside_disk = ~sat_disk
        
        time = np.array(data.time, dtype=float)
        
                
            
         
            
            
        for i,t2 in enumerate(time[1:]): # iterating over all recorded timesteps for the particle.
                i += 1
                    
                if ( and sat_disk[i]):
                    _in_ = data[time==t2].copy()
                    dsrg_accreted = pd.concat([dsrg_accreted, _in_])
                 

                
                
                
                
                
    # apply the calc_angles function along the rows of discharged particles.
   
    
    if save:
        filepath = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/accreted_test.hdf5'
        print(f'Saving {key} accreted particle dataset to {filepath}')
        dsrg_accreted.to_hdf(filepath, key=key)
        
        
    print(f'> Returning (dsrg_accreted) datasets <')

    return dsrg_accreted

### Advanced Accretion Calculation: 
______________________

In [7]:
def read_discharged(sim, haloid, verbose=False):
    discharged = pd.DataFrame()
    
    key = f'{sim}_{str(int(haloid))}'
    
    path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/discharged_particles.hdf5'
    discharged = pd.read_hdf(path, key=key)
    
    path = '/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/dsrg_accreted_particles.hdf5'
    accreted = pd.read_hdf(path, key=key)
    
    return discharged, accreted

In [272]:
sim = 'h148'
haloid = 13
verbose = False

###############################################################################

import tqdm

discharged, accreted = read_discharged(sim, haloid, verbose=verbose)  

if verbose: print(f'Now computing discharged particles for {sim}-{haloid}...')
adv_accreted = pd.DataFrame() # gas accreted following a discharge event.

pids = np.unique(discharged.pid) # quicker to use discharge bc unique.

recycleTime = {'recycleTime': ""}
accreted = accreted.join(pd.DataFrame(columns=recycleTime))

for pid in tqdm.tqdm(pids):
    dis = discharged[discharged.pid==pid]
    acc = accreted[accreted.pid==pid]
    
    dTime = np.asarray(dis.time)
    aTime = np.asarray(acc.time)

    
    # Case 1: removing initial accretion event if it does not correspond to a discharge; else no alterations.
    # check to ensure that our discharge event actually has an accretion.
    if (len(aTime) == 0) or (len(dTime) == 0): # if no instances of accretion or no instances of discharge, move on to new particle
        continue
    if (aTime[0] < dTime[0]):
        aCache = acc[1:]
        
    else:
        aCache = acc
    
    if len(aCache) == 0: # if no instances of reaccretion, move on to next particle
        continue
        
    dCache = dis[0:len(aCache)]
       
    aCache['recycleTime'] = np.array(aCache['time']) - np.array(dCache['time'])
    print(dCache['time'],aCache['time'], aCache['recycleTime'])

    """
    recycleTime = pd.DataFrame([])
    for i in range(0, len(aCache)):
        i+=1
        dt = aCache.time.iloc[i] - dCache.time.iloc[i-1]

        recycleTime = pd.concat([recycleTime, dt])   
    
    print(recycleTime)
    # caching time between discharge and the recorded accretion.
    aCache['recycleTime'] = recycleTime
    """
    adv_accreted = pd.concat([adv_accreted, aCache])

  1%|▍                                       | 55/5276 [00:00<00:19, 274.47it/s]

190920    11.214915
Name: time, dtype: float64 205282    11.645896
Name: time, dtype: float64 205282    0.43098
Name: recycleTime, dtype: float64
161854    10.7671
Name: time, dtype: float64 161854    10.7671
Name: time, dtype: float64 161854    0.0
Name: recycleTime, dtype: float64
161858    10.7671
Name: time, dtype: float64 161858    10.7671
Name: time, dtype: float64 161858    0.0
Name: recycleTime, dtype: float64
147226    10.352954
Name: time, dtype: float64 219418    12.076876
Name: time, dtype: float64 219418    1.723922
Name: recycleTime, dtype: float64
190945    11.214915
Name: time, dtype: float64 233230    12.150951
Name: time, dtype: float64 233230    0.936036
Name: recycleTime, dtype: float64
117820    9.490994
Name: time, dtype: float64 117820    9.490994
Name: time, dtype: float64 117820    0.0
Name: recycleTime, dtype: float64
205311    11.645896
Name: time, dtype: float64 260553    12.938837
Name: time, dtype: float64 260553    1.292941
Name: recycleTime, dtype: float

  2%|▊                                      | 112/5276 [00:00<00:19, 261.47it/s]

14974     6.905111
132749    9.921974
Name: time, dtype: float64 14974     6.905111
132749    9.921974
Name: time, dtype: float64 14974     0.0
132749    0.0
Name: recycleTime, dtype: float64
118040     9.490994
162094    10.767100
Name: time, dtype: float64 132750     9.921974
176631    10.783935
Name: time, dtype: float64 132750    0.430980
176631    0.016835
Name: recycleTime, dtype: float64
103328    9.060013
Name: time, dtype: float64 103328    9.060013
Name: time, dtype: float64 103328    0.0
Name: recycleTime, dtype: float64
59164    7.767072
88613    8.608831
Name: time, dtype: float64 59164    7.767072
88613    8.608831
Name: time, dtype: float64 59164    0.0
88613    0.0
Name: recycleTime, dtype: float64
14981    6.905111
Name: time, dtype: float64 59165    7.767072
Name: time, dtype: float64 59165    0.861961
Name: recycleTime, dtype: float64
14982    6.905111
Name: time, dtype: float64 44438    7.336092
Name: time, dtype: float64 44438    0.43098
Name: recycleTime, dtype: f

  3%|█                                      | 140/5276 [00:00<00:19, 266.29it/s]

...

57945    7.336092
Name: time, dtype: float64 57945    7.336092
Name: time, dtype: float64 57945    0.0
Name: recycleTime, dtype: float64
57956    7.336092
Name: time, dtype: float64 57956    7.336092
Name: time, dtype: float64 57956    0.0
Name: recycleTime, dtype: float64
57957    7.336092
Name: time, dtype: float64 57957    7.336092
Name: time, dtype: float64 57957    0.0
Name: recycleTime, dtype: float64
146257    9.921974
Name: time, dtype: float64 146257    9.921974
Name: time, dtype: float64 146257    0.0
Name: recycleTime, dtype: float64
28541    6.905111
Name: time, dtype: float64 28541    6.905111
Name: time, dtype: float64 28541    0.0
Name: recycleTime, dtype: float64
102165    8.608831
Name: time, dtype: float64 146288    9.921974
Name: time, dtype: float64 146288    1.313143
Name: recycleTime, dtype: float64
28555    6.905111
58011    7.336092
Name: time, dtype: float64 28555    6.905111
58011    7.336092
Name: time, dtype: float64 28555    0.0
58011    0.0
Name: recycleTi

 98%|█████████████████████████████████████▏| 5168/5276 [00:13<00:00, 434.72it/s]

102423    8.608831
Name: time, dtype: float64 102423    8.608831
Name: time, dtype: float64 102423    0.0
Name: recycleTime, dtype: float64
131856    9.490994
Name: time, dtype: float64 131856    9.490994
Name: time, dtype: float64 131856    0.0
Name: recycleTime, dtype: float64
102454    8.608831
131878    9.490994
Name: time, dtype: float64 102454    8.608831
146577    9.921974
Name: time, dtype: float64 102454    0.00000
146577    0.43098
Name: recycleTime, dtype: float64
43678    7.039793
Name: time, dtype: float64 43678    7.039793
Name: time, dtype: float64 43678    0.0
Name: recycleTime, dtype: float64
102605     8.608831
175917    10.767100
218934    11.645896
Name: time, dtype: float64 117319     9.060013
190450    10.783935
273332    12.938837
Name: time, dtype: float64 117319    0.451183
190450    0.016835
273332    1.292941
Name: recycleTime, dtype: float64
28995    6.905111
73179    7.767072
Name: time, dtype: float64 43723    7.039793
73179    7.767072
Name: time, dtype: 

100%|██████████████████████████████████████| 5276/5276 [00:13<00:00, 378.72it/s]

43961      7.039793
88138      8.198053
117570     9.060013
176165    10.767100
Name: time, dtype: float64 43961      7.039793
88138      8.198053
132280     9.490994
219178    11.645896
Name: time, dtype: float64 43961     0.000000
88138     0.000000
132280    0.430980
219178    0.878796
Name: recycleTime, dtype: float64
43962     7.039793
73418     7.767072
102857    8.608831
Name: time, dtype: float64 43962     7.039793
73418     7.767072
102857    8.608831
Name: time, dtype: float64 43962     0.0
73418     0.0
102857    0.0
Name: recycleTime, dtype: float64
43970     7.039793
73426     7.767072
102865    8.608831
Name: time, dtype: float64 43970     7.039793
73426     7.767072
146988    9.921974
Name: time, dtype: float64 43970     0.000000
73426     0.000000
146988    1.313143
Name: recycleTime, dtype: float64
73439    7.767072
Name: time, dtype: float64 73439    7.767072
Name: time, dtype: float64 73439    0.0
Name: recycleTime, dtype: float64
88174    8.198053
Name: time, dtype:

In [159]:
len(accreted)-len(adv_accreted)

2367

So we immediately see by this first condition on the *accreted* set that we remove a good number of particles not corresponding to a discharge event.

In [258]:
from accretion import *

def read_accreted():
    adv_accreted = pd.DataFrame()

    keys = get_keys()

    for i,key in enumerate(keys):
        i += 1
        sim = key[:4]
        haloid = int(key[5:])
        adv_accreted1 = pd.read_hdf('/home/lonzaric/astro_research/Stellar_Feedback_Code/SNeData/adv_accreted.hdf5',\
             key=key)
        adv_accreted1['key'] = key
        adv_accreted = pd.concat([adv_accreted, adv_accreted1])


    print(f'> Returning (adv. accreted) for all available satellites <')
    return adv_accreted


In [259]:
adv_accreted = read_accreted()

h148_13 h148_278 h148_28 h148_283 h148_329 h148_37 h148_45 h148_68 h148_80 h229_20 h229_22 h229_23 h229_27 h229_55 h242_24 h242_41 h242_80 h329_137 h329_33
> Returning (adv. accreted) for all available satellites <


In [260]:
adv_accreted

,time,pid,rho,temp,mass,coolontime,r,r_per_Rvir,x,y,...,host_r_gal,sat_disk,sat_halo,host_disk,host_halo,other_sat,IGM,t,sat_Mvir,key
205282,11.645896,2202002,0.002334,14873.835938,28366.657652,10.908183,2.917399,0.051277,-1.943967,-1.597916,...,21.516602,False,True,False,False,False,False,11.645896,9.672413e+09,h148_13
161854,10.767100,4073949,0.058975,9752.045898,26835.758536,0.000000,0.849211,0.015786,-0.681446,-0.438338,...,20.143633,False,True,False,False,False,False,10.767100,1.002284e+10,h148_13
161858,10.767100,4340176,0.057253,9207.494141,27066.785805,10.685118,0.846655,0.015738,-0.573782,-0.505544,...,20.143633,False,True,False,False,False,False,10.767100,1.002284e+10,h148_13
219418,12.076876,6936050,0.076830,8806.372070,27651.989951,11.744889,0.534446,0.009112,0.530554,0.043023,...,18.811293,False,True,False,False,False,False,12.076876,9.621392e+09,h148_13
233230,12.150951,6988228,0.000113,170371.390625,26850.013274,11.038656,5.612292,0.095189,4.317860,2.915293,...,20.529471,False,False,False,False,False,True,12.150951,9.613553e+09,h148_13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26198,8.608831,6627712,0.033717,12044.281250,26635.610201,4.748210,1.267604,0.110037,-0.912659,-0.696403,...,9.596679,False,True,False,False,False,False,8.608831,1.604989e+09,h329_33
21829,8.413543,6627722,0.024180,12440.765625,26519.316821,0.000000,1.900877,0.063894,-1.505597,-0.240243,...,10.988910,False,True,False,False,False,False,8.413543,3.091517e+09,h329_33
8719,7.443837,6627756,0.066316,11737.336914,26526.461460,0.000000,1.097153,0.040304,0.885839,0.543622,...,12.407557,False,True,False,False,False,False,7.443837,3.140099e+09,h329_33
26208,8.608831,6627757,0.014028,13289.551758,26584.661687,0.000000,2.624332,0.227811,-1.626817,1.831154,...,9.596679,False,True,False,False,False,False,8.608831,1.604989e+09,h329_33
